In [ ]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import os
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 150

In [ ]:
load_dotenv()

In [ ]:
conn_string = (
    f"host={os.getenv('hostname')} "
    f"dbname={os.getenv('database')} "
    f"user={os.getenv('username')} "
    f"password={os.getenv('password')} "
    f"port={os.getenv('port')}"
)

In [ ]:
conn = psycopg2.connect(conn_string)

In [ ]:
cursor = conn.cursor()

In [ ]:
schema_name = "public,dadeh"

In [ ]:
cursor.execute(f"SET search_path TO {schema_name}")

In [ ]:
df = pd.read_csv("contrat_inseres_employeurs.csv", dtype=str)

In [ ]:
assures = list(df["id_assure"].unique())

In [ ]:
len(assures)

In [ ]:
dfa = pd.DataFrame([])

In [ ]:
i = 0
cpt = 0
for ass in assures:
    if cpt == 0:
        str_as = ""
    cpt += 1
    str_as += "id = " + ass + " OR "
    if cpt == 100:
        # send
        str_as = str_as[:-3]
        query = (
            "select id, num_immatriculation, date_naissance, code_departement_naissance "
            "from ddadtassure where " + str_as
        )
        dfinter = pd.read_sql_query(query, conn)
        dfa = pd.concat([dfa, dfinter])
        cpt = 0
        i = i + 1
        print(i * 100)

In [ ]:
dfa

In [ ]:
dfa = dfa.rename(columns={"id": "id_assure"})

In [ ]:
dfa.to_csv("assures_inseres.csv", index=False)

In [ ]:
dfa = pd.read_csv("assures_inseres.csv", dtype=str)

In [ ]:
dfa

In [ ]:
df = pd.merge(df, dfa, on="id_assure", how="left")

In [ ]:
df["sexe_assure"] = df["num_immatriculation"].apply(lambda x: x[0] if x == x else None)

In [ ]:
df.to_csv("contrat_inseres_employeurs_assures.csv", index=False)

In [ ]:
df = pd.read_csv("contrat_inseres_employeurs_assures.csv", dtype=str)

In [ ]:
employeursassures = list(df["id_employeur_assure"].unique())

In [ ]:
len(employeursassures)

In [ ]:
dfa = pd.DataFrame([])

In [ ]:
df = pd.merge(df, dfa, on="id_employeur_assure", how="left")

In [ ]:
df.to_csv("complet_J1_1850.csv", index=False)